<a href="https://colab.research.google.com/github/koalalovepabro/Stock-Portfolio-Optimization/blob/main/Build_A_Killer_Stock_Portfolio(KRX).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 최적화된 주식 포트폴리오 만들기

## 💘 한국 주식 적용하기

In [1]:
!pip install -q install finance-datareader

In [2]:
import pandas as pd
import numpy as np
import requests

In [3]:
# Finance Datareader 불러오고, 버전 확인하기
import FinanceDataReader as fdr
fdr.__version__

'0.9.31'

In [4]:
# KRX : KRX 종목 전체
# KOSPI : KOSPI 종목
# KOSDAQ : KOSDAQ 종목
# KONEX : KONEX 종목
# NASDAQ : 나스닥 종목
# NYSE : 뉴욕증권거래소 종목
# SP500 : S&P500 종목

In [5]:
# 한국거래소(KRX) 상장종목 전체 가져오기
df_krx = fdr.StockListing('KRX')
df_krx

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,특수 목적용 기계 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,"박종익, 김세완 (각자 대표이사)",http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,박대현,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도
...,...,...,...,...,...,...,...,...,...,...
6460,000547,KOSPI,흥국화재2우B,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6461,000545,KOSPI,흥국화재우,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6462,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),1976-06-29,12월,이환구,http://www.heung-a.com,서울특별시
6463,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,2000-02-03,12월,"정영균, 이목운, 허철호 (각자대표)",http://www.heerim.com,서울특별시


In [6]:
# 데이터 파악
# 데이터의 결측치 존재 여부, 데이터의 타입, 컬럼의 수, 데이터프레임의 메모리 사용량 등
df_krx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6465 entries, 0 to 6464
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Symbol          6465 non-null   object        
 1   Market          6465 non-null   object        
 2   Name            6465 non-null   object        
 3   Sector          2450 non-null   object        
 4   Industry        2431 non-null   object        
 5   ListingDate     2450 non-null   datetime64[ns]
 6   SettleMonth     2450 non-null   object        
 7   Representative  2450 non-null   object        
 8   HomePage        2269 non-null   object        
 9   Region          2450 non-null   object        
dtypes: datetime64[ns](1), object(9)
memory usage: 555.6+ KB


In [7]:
# 데이터의 결측치 확인
df_krx.isnull().sum()

Symbol               0
Market               0
Name                 0
Sector            4015
Industry          4034
ListingDate       4015
SettleMonth       4015
Representative    4015
HomePage          4196
Region            4015
dtype: int64

In [8]:
# 결측치 제거
df_krx = df_krx.dropna()
df_krx.isnull().sum()

Symbol            0
Market            0
Name              0
Sector            0
Industry          0
ListingDate       0
SettleMonth       0
Representative    0
HomePage          0
Region            0
dtype: int64

In [9]:
df_krx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2256 entries, 0 to 6464
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Symbol          2256 non-null   object        
 1   Market          2256 non-null   object        
 2   Name            2256 non-null   object        
 3   Sector          2256 non-null   object        
 4   Industry        2256 non-null   object        
 5   ListingDate     2256 non-null   datetime64[ns]
 6   SettleMonth     2256 non-null   object        
 7   Representative  2256 non-null   object        
 8   HomePage        2256 non-null   object        
 9   Region          2256 non-null   object        
dtypes: datetime64[ns](1), object(9)
memory usage: 193.9+ KB


In [10]:
# 데이터프레임의 통계정보
df_krx.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
count,2256,2256,2256,2256,2256,2256,2256,2256,2256,2256
unique,2256,3,2256,158,2163,1699,9,2158,2237,22
top,237690,KOSDAQ,한세엠케이,특수 목적용 기계 제조업,지주회사,1976-06-30 00:00:00,12월,-,http://www.moorim.co.kr,서울특별시
freq,1,1352,1,135,19,8,2205,5,2,888
first,NaN,NaN,NaN,NaN,NaN,1956-03-03 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,2021-07-16 00:00:00,NaN,NaN,NaN,NaN


In [11]:
# 종목코드 가져오기
assets = df_krx['Symbol']
assets

0       060310
1       095570
2       006840
3       054620
4       265520
         ...  
6458    189980
6459    000540
6462    003280
6463    037440
6464    238490
Name: Symbol, Length: 2256, dtype: object

In [12]:
assets = np.array(assets)
assets

array(['060310', '095570', '006840', ..., '003280', '037440', '238490'],
      dtype=object)

In [13]:
len(assets)

2256

In [16]:
# 종목별 종가 가져오기
from datetime import datetime

# 주식 시작일은 2013년 1월 1일이고 종료일은 현재 날짜 (오늘)로 설정
#Get the stock starting date
start_date = '2013-01-01'
# today = datetime.today().strftime('%Y-%m-%d')
end_date = '2021-07-15'

# 각 주식의 일별 종가 데이터를 저장할 데이터 프레임을 생성
#Create a dataframe to store the adjusted close price of the stocks
df = pd.DataFrame()

# FinanceDataReader로 각 종목의 종가데이터 불러오기
for stock in assets:
  df[stock] = fdr.DataReader(stock, start_date, end_date)['Close']
df

,060310,095570,006840,054620,265520,211270,027410,282330,138930,001460,013720,001040,079160,035760,000120,011150,097950,051500,058820,023460,056730,056000,065770,083660,000590,012030,016610,005830,000990,139130,001530,000210,001880,068790,004840,241520,155660,069730,017940,245620,...,010690,126640,133820,061250,010660,000850,016580,032560,004800,094280,298040,298050,298020,298000,093370,050090,081660,290270,005870,284420,090710,205470,028080,032860,200670,212310,079980,065510,005010,243070,084110,263920,145020,069260,010240,189980,000540,003280,037440,238490
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-02,7150,NaN,21565,5722,NaN,NaN,NaN,NaN,12536,186000,3040,117399,29253,273700,103500,3015,362000,35900,1212,1295,5822,1880,4940,2143,33900,638,3825,44450,6400,14366,38245,79970,3437,2930,6071,NaN,NaN,2955,69200,NaN,...,9980,1320,NaN,2483,2400,43100,9040,3378,43970,4200,NaN,NaN,NaN,NaN,4862,1275,12680,NaN,3865,NaN,805,NaN,6610,394737,NaN,NaN,10750,14850,25100,NaN,9043,NaN,NaN,25600,1928,NaN,4915,6352,6730,NaN
2013-01-03,7090,NaN,22292,6035,NaN,NaN,NaN,NaN,12816,185500,3085,116456,29117,274000,102000,2970,369000,35700,1176,1270,5804,1900,4830,2128,33150,632,3830,43800,6450,14221,38377,82861,3511,2920,6003,NaN,NaN,2950,68000,NaN,...,9940,1300,NaN,2600,2415,43500,9380,3391,44266,4230,NaN,NaN,NaN,NaN,4852,1280,12220,NaN,3860,NaN,816,NaN,6710,395729,NaN,NaN,10600,14200,25650,NaN,9141,NaN,NaN,25500,1925,NaN,4965,6345,6770,NaN
2013-01-04,7330,NaN,22971,6296,NaN,NaN,NaN,NaN,12955,184000,3010,120700,29299,270500,107500,3030,375000,35600,1144,1255,5804,1890,4840,2074,32950,634,3830,43600,6470,14414,38333,82229,3627,3025,6208,NaN,NaN,2950,68000,NaN,...,9890,1300,NaN,2990,2450,44300,9450,3395,44207,4305,NaN,NaN,NaN,NaN,4877,1280,12200,NaN,3830,NaN,810,NaN,6600,396721,NaN,NaN,10300,14500,25300,NaN,9173,NaN,NaN,25050,1925,NaN,4900,6572,6700,NaN
2013-01-07,7850,NaN,23068,6236,NaN,NaN,NaN,NaN,13049,183500,3275,119757,29707,273000,107500,2985,368000,35050,1171,1245,5877,1910,4850,2060,33200,647,3820,44500,6920,14366,38910,81054,3627,3175,5952,NaN,NaN,2940,67100,NaN,...,9810,1305,NaN,3020,2460,43450,9470,3439,43437,4470,NaN,NaN,NaN,NaN,4975,1270,12260,NaN,3840,NaN,821,NaN,6620,396721,NaN,NaN,10400,14850,25350,NaN,9371,NaN,NaN,25650,1915,NaN,4905,6696,6680,NaN
2013-01-08,8960,NaN,22971,6236,NaN,NaN,NaN,NaN,13142,185000,3195,115513,29616,275500,113000,2930,358000,35150,1171,1255,5949,1885,4900,2055,33150,652,3825,42800,7060,14655,39133,79157,3733,3230,5934,NaN,NaN,2970,69100,NaN,...,10200,1320,NaN,3033,2455,43450,9500,3456,42076,4420,NaN,NaN,NaN,NaN,5024,1265,12480,NaN,3800,NaN,804,NaN,6500,396721,NaN,NaN,10350,15200,25500,NaN,9109,NaN,NaN,26250,1923,NaN,4910,6696,6720,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-09,2955,5850.0,27650,14850,29300.0,18150.0,6160.0,163000.0,7350,390500,22750,98400,29400,179800,181500,5040,442500,32750,4975,3455,1550,2860,4900,1695,67800,1225,6720,53500,60600,8810,195000,77300,35450,7470,7590,5500.0,6680.0,6700,55400,5480.0,...,10050,2485,2795.0,3930,2755,37350,22150,9120,104500,21850,70100.0,513000.0,890000.0,381500.0,11450,1330,53000,7560.0,8040,18300.0,998,25550.0,5680,6850,37800.0,3200.0,10650,9300,13800,64000.0,74300,4555.0,231500.0,22400,11850,4235.0,4340,1035,7420,10950.0
2021-07-12,2940,5910.0,28550,14850,29650.0,17900.0,6260.0,167000.0,7520,395000,22750,101000,29900,180000,181000,4965,443000,32750,4915,3450,1550,2860,4980,1675,68500,1255,6980,55000,64300,9030,195500,79200,35850,7570,7610,5520.0,6870.0,6630,56000,5400.0,...,9880,2555,2970.0,3925,2760,37800,21950,9260,108500,22200,71400.0,514000.0,878000.0,379000.0,11900,1325,53400,7620.0,8130,18300.0,1075,24750.0,5830,6600,37

In [17]:
df

,060310,095570,006840,054620,265520,211270,027410,282330,138930,001460,013720,001040,079160,035760,000120,011150,097950,051500,058820,023460,056730,056000,065770,083660,000590,012030,016610,005830,000990,139130,001530,000210,001880,068790,004840,241520,155660,069730,017940,245620,...,010690,126640,133820,061250,010660,000850,016580,032560,004800,094280,298040,298050,298020,298000,093370,050090,081660,290270,005870,284420,090710,205470,028080,032860,200670,212310,079980,065510,005010,243070,084110,263920,145020,069260,010240,189980,000540,003280,037440,238490
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-02,7150,NaN,21565,5722,NaN,NaN,NaN,NaN,12536,186000,3040,117399,29253,273700,103500,3015,362000,35900,1212,1295,5822,1880,4940,2143,33900,638,3825,44450,6400,14366,38245,79970,3437,2930,6071,NaN,NaN,2955,69200,NaN,...,9980,1320,NaN,2483,2400,43100,9040,3378,43970,4200,NaN,NaN,NaN,NaN,4862,1275,12680,NaN,3865,NaN,805,NaN,6610,394737,NaN,NaN,10750,14850,25100,NaN,9043,NaN,NaN,25600,1928,NaN,4915,6352,6730,NaN
2013-01-03,7090,NaN,22292,6035,NaN,NaN,NaN,NaN,12816,185500,3085,116456,29117,274000,102000,2970,369000,35700,1176,1270,5804,1900,4830,2128,33150,632,3830,43800,6450,14221,38377,82861,3511,2920,6003,NaN,NaN,2950,68000,NaN,...,9940,1300,NaN,2600,2415,43500,9380,3391,44266,4230,NaN,NaN,NaN,NaN,4852,1280,12220,NaN,3860,NaN,816,NaN,6710,395729,NaN,NaN,10600,14200,25650,NaN,9141,NaN,NaN,25500,1925,NaN,4965,6345,6770,NaN
2013-01-04,7330,NaN,22971,6296,NaN,NaN,NaN,NaN,12955,184000,3010,120700,29299,270500,107500,3030,375000,35600,1144,1255,5804,1890,4840,2074,32950,634,3830,43600,6470,14414,38333,82229,3627,3025,6208,NaN,NaN,2950,68000,NaN,...,9890,1300,NaN,2990,2450,44300,9450,3395,44207,4305,NaN,NaN,NaN,NaN,4877,1280,12200,NaN,3830,NaN,810,NaN,6600,396721,NaN,NaN,10300,14500,25300,NaN,9173,NaN,NaN,25050,1925,NaN,4900,6572,6700,NaN
2013-01-07,7850,NaN,23068,6236,NaN,NaN,NaN,NaN,13049,183500,3275,119757,29707,273000,107500,2985,368000,35050,1171,1245,5877,1910,4850,2060,33200,647,3820,44500,6920,14366,38910,81054,3627,3175,5952,NaN,NaN,2940,67100,NaN,...,9810,1305,NaN,3020,2460,43450,9470,3439,43437,4470,NaN,NaN,NaN,NaN,4975,1270,12260,NaN,3840,NaN,821,NaN,6620,396721,NaN,NaN,10400,14850,25350,NaN,9371,NaN,NaN,25650,1915,NaN,4905,6696,6680,NaN
2013-01-08,8960,NaN,22971,6236,NaN,NaN,NaN,NaN,13142,185000,3195,115513,29616,275500,113000,2930,358000,35150,1171,1255,5949,1885,4900,2055,33150,652,3825,42800,7060,14655,39133,79157,3733,3230,5934,NaN,NaN,2970,69100,NaN,...,10200,1320,NaN,3033,2455,43450,9500,3456,42076,4420,NaN,NaN,NaN,NaN,5024,1265,12480,NaN,3800,NaN,804,NaN,6500,396721,NaN,NaN,10350,15200,25500,NaN,9109,NaN,NaN,26250,1923,NaN,4910,6696,6720,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-09,2955,5850.0,27650,14850,29300.0,18150.0,6160.0,163000.0,7350,390500,22750,98400,29400,179800,181500,5040,442500,32750,4975,3455,1550,2860,4900,1695,67800,1225,6720,53500,60600,8810,195000,77300,35450,7470,7590,5500.0,6680.0,6700,55400,5480.0,...,10050,2485,2795.0,3930,2755,37350,22150,9120,104500,21850,70100.0,513000.0,890000.0,381500.0,11450,1330,53000,7560.0,8040,18300.0,998,25550.0,5680,6850,37800.0,3200.0,10650,9300,13800,64000.0,74300,4555.0,231500.0,22400,11850,4235.0,4340,1035,7420,10950.0
2021-07-12,2940,5910.0,28550,14850,29650.0,17900.0,6260.0,167000.0,7520,395000,22750,101000,29900,180000,181000,4965,443000,32750,4915,3450,1550,2860,4980,1675,68500,1255,6980,55000,64300,9030,195500,79200,35850,7570,7610,5520.0,6870.0,6630,56000,5400.0,...,9880,2555,2970.0,3925,2760,37800,21950,9260,108500,22200,71400.0,514000.0,878000.0,379000.0,11900,1325,53400,7620.0,8130,18300.0,1075,24750.0,5830,6600,37

In [22]:
# DataFrame을 csv 파일로 저장하기 ( 결측값 제거하지 않음 )
df.to_csv("krx_code_close.csv", index=True)

In [29]:
# 칼럼명을 회사이름으로 변경
df.columns = df_krx['Name'].values
df

,3S,AJ네트웍스,AK홀딩스,APS홀딩스,AP시스템,AP위성,BGF,BGF리테일,BNK금융지주,BYC,CBI,CJ,CJ CGV,CJ ENM,CJ대한통운,CJ씨푸드,CJ제일제당,CJ프레시웨이,CMG제약,CNH,CNT85,COWON,CS,CSA 코스믹,CS홀딩스,DB,DB금융투자,DB손해보험,DB하이텍,DGB금융지주,DI동일,DL,DL건설,DMS,DRB동일,DSC인베스트먼트,DSR,DSR제강,E1,EDGC,...,화신,화신정공,화인베스틸,화일약품,화천기계,화천기공,환인제약,황금에스티,효성,효성ITX,효성중공업,효성첨단소재,효성티앤씨,효성화학,후성,휘닉스소재,휠라홀딩스,휴네시온,휴니드,휴럼,휴림로봇,휴마시스,휴맥스홀딩스,휴먼엔,휴메딕스,휴벡셀,휴비스,휴비츠,휴스틸,휴온스,휴온스글로벌,휴온스블러썸,휴젤,휴켐스,흥국,흥국에프엔비,흥국화재,흥아해운,희림,힘스
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-02,7150,NaN,21565,5722,NaN,NaN,NaN,NaN,12536,186000,3040,117399,29253,273700,103500,3015,362000,35900,1212,1295,5822,1880,4940,2143,33900,638,3825,44450,6400,14366,38245,79970,3437,2930,6071,NaN,NaN,2955,69200,NaN,...,9980,1320,NaN,2483,2400,43100,9040,3378,43970,4200,NaN,NaN,NaN,NaN,4862,1275,12680,NaN,3865,NaN,805,NaN,6610,394737,NaN,NaN,10750,14850,25100,NaN,9043,NaN,NaN,25600,1928,NaN,4915,6352,6730,NaN
2013-01-03,7090,NaN,22292,6035,NaN,NaN,NaN,NaN,12816,185500,3085,116456,29117,274000,102000,2970,369000,35700,1176,1270,5804,1900,4830,2128,33150,632,3830,43800,6450,14221,38377,82861,3511,2920,6003,NaN,NaN,2950,68000,NaN,...,9940,1300,NaN,2600,2415,43500,9380,3391,44266,4230,NaN,NaN,NaN,NaN,4852,1280,12220,NaN,3860,NaN,816,NaN,6710,395729,NaN,NaN,10600,14200,25650,NaN,9141,NaN,NaN,25500,1925,NaN,4965,6345,6770,NaN
2013-01-04,7330,NaN,22971,6296,NaN,NaN,NaN,NaN,12955,184000,3010,120700,29299,270500,107500,3030,375000,35600,1144,1255,5804,1890,4840,2074,32950,634,3830,43600,6470,14414,38333,82229,3627,3025,6208,NaN,NaN,2950,68000,NaN,...,9890,1300,NaN,2990,2450,44300,9450,3395,44207,4305,NaN,NaN,NaN,NaN,4877,1280,12200,NaN,3830,NaN,810,NaN,6600,396721,NaN,NaN,10300,14500,25300,NaN,9173,NaN,NaN,25050,1925,NaN,4900,6572,6700,NaN
2013-01-07,7850,NaN,23068,6236,NaN,NaN,NaN,NaN,13049,183500,3275,119757,29707,273000,107500,2985,368000,35050,1171,1245,5877,1910,4850,2060,33200,647,3820,44500,6920,14366,38910,81054,3627,3175,5952,NaN,NaN,2940,67100,NaN,...,9810,1305,NaN,3020,2460,43450,9470,3439,43437,4470,NaN,NaN,NaN,NaN,4975,1270,12260,NaN,3840,NaN,821,NaN,6620,396721,NaN,NaN,10400,14850,25350,NaN,9371,NaN,NaN,25650,1915,NaN,4905,6696,6680,NaN
2013-01-08,8960,NaN,22971,6236,NaN,NaN,NaN,NaN,13142,185000,3195,115513,29616,275500,113000,2930,358000,35150,1171,1255,5949,1885,4900,2055,33150,652,3825,42800,7060,14655,39133,79157,3733,3230,5934,NaN,NaN,2970,69100,NaN,...,10200,1320,NaN,3033,2455,43450,9500,3456,42076,4420,NaN,NaN,NaN,NaN,5024,1265,12480,NaN,3800,NaN,804,NaN,6500,396721,NaN,NaN,10350,15200,25500,NaN,9109,NaN,NaN,26250,1923,NaN,4910,6696,6720,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-09,2955,5850.0,27650,14850,29300.0,18150.0,6160.0,163000.0,7350,390500,22750,98400,29400,179800,181500,5040,442500,32750,4975,3455,1550,2860,4900,1695,67800,1225,6720,53500,60600,8810,195000,77300,35450,7470,7590,5500.0,6680.0,6700,55400,5480.0,...,10050,2485,2795.0,3930,2755,37350,22150,9120,104500,21850,70100.0,513000.0,890000.0,381500.0,11450,1330,53000,7560.0,8040,18300.0,998,25550.0,5680,6850,37800.0,3200.0,10650,9300,13800,64000.0,74300,4555.0,231500.0,22400,11850,4235.0,4340,1035,7420,10950.0
2021-07-12,2940,5910.0,28550,14850,29650.0,17900.0,6260.0,167000.0,7520,395000,22750,101000,29900,180000,181000,4965,443000,32750,4915,3450,1550,2860,4980,1675,68500,1255,6980,55000,64300,9030,195500,79200,35850,7570,7610,5520.0,6870.0,6630,56000,5400.0,...,9880,2555,2970.0,3925,2760,37800,21950,9260,108500,22200,71400.0,514000.0,878000.0,379000.0,11900,1325,53400,7620.0,8130,18300.0,1075,24750.0,5830,6600,37050.0,3390.0,10700,9540,14050,63300.0,71100,4555.0,228000.0,24250,11700,4290.0,4395,1035,7550,11100.0
2021-07-13,2930,5920.0,29050,15700,299

In [30]:
# 결측값 있는 열 삭제  ( 종목 2256 -> 1476으로 줄어 듦 )
df2 = df.dropna(axis = 1)
df2

,3S,AK홀딩스,APS홀딩스,BNK금융지주,BYC,CBI,CJ,CJ CGV,CJ ENM,CJ대한통운,CJ씨푸드,CJ제일제당,CJ프레시웨이,CMG제약,CNH,CNT85,COWON,CS,CSA 코스믹,CS홀딩스,DB,DB금융투자,DB손해보험,DB하이텍,DGB금융지주,DI동일,DL,DL건설,DMS,DRB동일,DSR제강,E1,EG,EMW,ES큐브,F&F홀딩스,GKL,GST,GS건설,GS글로벌,...,현대코퍼레이션,현대퓨처넷,현대해상,현대홈쇼핑,현진소재,형지I&C,형지엘리트,혜인,호텔신라,홈센타홀딩스,홈캐스트,화성밸브,화성산업,화승인더,화승코퍼레이션,화신,화신정공,화일약품,화천기계,화천기공,환인제약,황금에스티,효성,효성ITX,후성,휘닉스소재,휠라홀딩스,휴니드,휴림로봇,휴맥스홀딩스,휴먼엔,휴비스,휴비츠,휴스틸,휴온스글로벌,휴켐스,흥국,흥국화재,흥아해운,희림
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-02,7150,21565,5722,12536,186000,3040,117399,29253,273700,103500,3015,362000,35900,1212,1295,5822,1880,4940,2143,33900,638,3825,44450,6400,14366,38245,79970,3437,2930,6071,2955,69200,43650,3227,15370,3020,29250,4165,55297,8569,...,18735,4430,32000,122500,29351,916,2152,3555,43500,1341,4085,3224,4500,839,671,9980,1320,2483,2400,43100,9040,3378,43970,4200,4862,1275,12680,3865,805,6610,394737,10750,14850,25100,9043,25600,1928,4915,6352,6730
2013-01-03,7090,22292,6035,12816,185500,3085,116456,29117,274000,102000,2970,369000,35700,1176,1270,5804,1900,4830,2128,33150,632,3830,43800,6450,14221,38377,82861,3511,2920,6003,2950,68000,40750,3197,15120,2975,28550,4185,55297,8569,...,19131,4580,31750,120500,28870,940,2162,3600,41800,1349,4165,3171,4550,833,674,9940,1300,2600,2415,43500,9380,3391,44266,4230,4852,1280,12220,3860,816,6710,395729,10600,14200,25650,9141,25500,1925,4965,6345,6770
2013-01-04,7330,22971,6296,12955,184000,3010,120700,29299,270500,107500,3030,375000,35600,1144,1255,5804,1890,4840,2074,32950,634,3830,43600,6470,14414,38333,82229,3627,3025,6208,2950,68000,41000,3264,15480,3015,29700,4115,54095,8355,...,19087,4655,32650,123500,29031,948,2166,3620,41550,1322,4420,3145,4560,819,689,9890,1300,2990,2450,44300,9450,3395,44207,4305,4877,1280,12200,3830,810,6600,396721,10300,14500,25300,9173,25050,1925,4900,6572,6700
2013-01-07,7850,23068,6236,13049,183500,3275,119757,29707,273000,107500,2985,368000,35050,1171,1245,5877,1910,4850,2060,33200,647,3820,44500,6920,14366,38910,81054,3627,3175,5952,2940,67100,39800,3211,15170,3000,30300,4300,53263,8142,...,18911,4795,33150,125500,29431,1003,2213,3600,44000,1282,4570,3115,4450,827,686,9810,1305,3020,2460,43450,9470,3439,43437,4470,4975,1270,12260,3840,821,6620,396721,10400,14850,25350,9371,25650,1915,4905,6696,6680
2013-01-08,8960,22971,6236,13142,185000,3195,115513,29616,275500,113000,2930,358000,35150,1171,1255,5949,1885,4900,2055,33150,652,3825,42800,7060,14655,39133,79157,3733,3230,5934,2970,69100,39300,3227,15430,3025,29900,4295,52431,8213,...,19219,4760,32100,125000,28951,1019,2236,3585,43750,1313,5000,3149,4280,841,680,10200,1320,3033,2455,43450,9500,3456,42076,4420,5024,1265,12480,3800,804,6500,396721,10350,15200,25500,9109,26250,1923,4910,6696,6720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-09,2955,27650,14850,7350,390500,22750,98400,29400,179800,181500,5040,442500,32750,4975,3455,1550,2860,4900,1695,67800,1225,6720,53500,60600,8810,195000,77300,35450,7470,7590,6700,55400,13350,2780,6140,35950,16350,26000,42050,2815,...,19950,4325,24700,86200,400,1615,5190,6800,92700,1285,3435,7500,13400,7380,3150,10050,2485,3930,2755,37350,22150,9120,104500,21850,11450,1330,53000,8040,998,5680,6850,10650,9300,13800,74300,22400,11850,4340,1035,7420
2021-07-12,2940,28550,14850,7520,395000,22750,101000,29900,180000,181000,4965,443000,32750,4915,3450,1550,2860,4980,1675,68500,1255,6980,55000,64300,9030,195500,79200,35850,7570,7610,6630,56000,13350,2780,6290,36050,16450,26150,43600,2915,...,20000,4370,25300,87000,400,1600,5200,6950,93300,1315,3450,7720,13600,7560,3110,9880,2555,3925,2760,37800,21950,9260,108500,22200,11900,1325,53400,8130,1075,5830,6600,10700,9540,14050,71100,24250,11700,4395,1035,75

In [32]:
df2.isnull().sum()

3S         0
AK홀딩스      0
APS홀딩스     0
BNK금융지주    0
BYC        0
          ..
휴켐스        0
흥국         0
흥국화재       0
흥아해운       0
희림         0
Length: 1476, dtype: int64

In [33]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2101 entries, 2013-01-02 to 2021-07-15
Columns: 1476 entries, 3S to 희림
dtypes: int64(1476)
memory usage: 23.7 MB


In [34]:
# 결측값을 가진 열을 제거한 DataFrame을 csv 파일로 저장하기
df2.to_csv("krx_name_close_drop_columns.csv", index=True)

In [35]:
# Get the assets / tickers
assets = df2.columns
assets

Index(['3S', 'AK홀딩스', 'APS홀딩스', 'BNK금융지주', 'BYC', 'CBI', 'CJ', 'CJ CGV',
       'CJ ENM', 'CJ대한통운',
       ...
       '휴먼엔', '휴비스', '휴비츠', '휴스틸', '휴온스글로벌', '휴켐스', '흥국', '흥국화재', '흥아해운',
       '희림'],
      dtype='object', length=1476)

In [36]:
len(assets)

1476

In [37]:
df2.isnull().sum()

3S         0
AK홀딩스      0
APS홀딩스     0
BNK금융지주    0
BYC        0
          ..
휴켐스        0
흥국         0
흥국화재       0
흥아해운       0
희림         0
Length: 1476, dtype: int64

- Index : 날짜 (2,101  days)
- column : 종목명 (1,476개 종목)
- row : 날짜별 종목의 종가(Close Price)

In [38]:
!pip install PyPortfolioOpt

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 1.3MB 17.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for cvxpy: filename=cvxpy-1.1.13-cp37-cp37m-linux_x86_64.whl size=2736188 sha256=0daddd47c5be6ecee81a009072aa44cb44d55d796ead1d342cbb81544b82cda9
  Stored in directory: /root/.cache/pip/wheels/f9/78/00/f29636789ee83434953b5442f16ec3f9834a68e7fd0393c220
Successfully built cvxpy
  Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [39]:
# Optimize the portfolio
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [40]:
# Calculate the expected annualized returns and the annualized sample covariance matrix of the daily asset returns
mu = expected_returns.mean_historical_return(df2)
S = risk_models.sample_cov(df2)

In [41]:
mu

3S        -0.101338
AK홀딩스      0.036614
APS홀딩스     0.147432
BNK금융지주   -0.058883
BYC        0.096404
             ...   
휴켐스       -0.008212
흥국         0.239644
흥국화재      -0.012792
흥아해운      -0.195653
희림         0.007629
Length: 1476, dtype: float64

In [42]:
S

,3S,AK홀딩스,APS홀딩스,BNK금융지주,BYC,CBI,CJ,CJ CGV,CJ ENM,CJ대한통운,CJ씨푸드,CJ제일제당,CJ프레시웨이,CMG제약,CNH,CNT85,COWON,CS,CSA 코스믹,CS홀딩스,DB,DB금융투자,DB손해보험,DB하이텍,DGB금융지주,DI동일,DL,DL건설,DMS,DRB동일,DSR제강,E1,EG,EMW,ES큐브,F&F홀딩스,GKL,GST,GS건설,GS글로벌,...,현대코퍼레이션,현대퓨처넷,현대해상,현대홈쇼핑,현진소재,형지I&C,형지엘리트,혜인,호텔신라,홈센타홀딩스,홈캐스트,화성밸브,화성산업,화승인더,화승코퍼레이션,화신,화신정공,화일약품,화천기계,화천기공,환인제약,황금에스티,효성,효성ITX,후성,휘닉스소재,휠라홀딩스,휴니드,휴림로봇,휴맥스홀딩스,휴먼엔,휴비스,휴비츠,휴스틸,휴온스글로벌,휴켐스,흥국,흥국화재,흥아해운,희림
3S,0.299297,0.045523,0.049292,0.016259,0.020442,0.025560,0.026486,0.043403,0.018350,0.012070,0.031219,0.019358,0.026638,0.053949,0.034266,0.013252,0.029998,0.044681,0.014225,0.017681,0.033101,0.038376,0.011778,0.037747,0.016626,0.030028,0.032293,0.028970,0.051919,0.029784,0.036611,0.020277,0.043585,0.024173,0.046026,0.031475,0.032814,0.045943,0.037507,0.042643,...,0.054548,0.026175,0.006495,0.016637,0.049175,0.049805,0.051366,0.039268,0.034494,0.030234,0.064590,0.043687,0.027440,0.036917,0.035792,0.031828,0.041416,0.038989,0.033106,0.018001,0.021439,0.041062,0.026053,0.041004,0.049452,0.048034,0.030248,0.018576,0.053309,0.024696,0.041407,0.028491,0.036144,0.023056,0.039794,0.027589,0.047070,0.029733,0.040025,0.029341
AK홀딩스,0.045523,0.180536,0.044027,0.028559,0.013711,0.033646,0.038635,0.054883,0.030432,0.019493,0.030498,0.035239,0.048444,0.042580,0.023618,0.009634,0.047498,0.031917,0.018590,0.016339,0.033005,0.038813,0.026967,0.038340,0.027765,0.032351,0.034270,0.027971,0.046524,0.036980,0.046721,0.018921,0.040598,0.012899,0.037154,0.044310,0.046400,0.052530,0.036746,0.043566,...,0.046764,0.024164,0.020857,0.029290,0.049410,0.029890,0.036850,0.029929,0.043885,0.028156,0.047389,0.040397,0.033060,0.044669,0.033177,0.034791,0.040181,0.043339,0.024358,0.020553,0.030812,0.044444,0.033764,0.030137,0.046715,0.043396,0.039161,0.029823,0.051557,0.023962,0.040260,0.031615,0.038504,0.027629,0.049137,0.031881,0.037078,0.036007,0.048133,0.017887
APS홀딩스,0.049292,0.044027,0.274046,0.018321,0.022236,0.044325,0.037217,0.050242,0.035039,0.022649,0.034507,0.029422,0.040279,0.052729,0.029186,0.027111,0.040039,0.054840,0.016856,0.021638,0.035887,0.042840,0.014252,0.064758,0.020428,0.034383,0.034721,0.029463,0.085315,0.034337,0.053692,0.023278,0.038292,0.019849,0.052348,0.032600,0.038206,0.077953,0.046215,0.041666,...,0.050679,0.030074,0.008693,0.020987,0.051832,0.054172,0.046130,0.041847,0.034505,0.042950,0.071323,0.049096,0.032320,0.058693,0.041720,0.037174,0.054597,0.049346,0.043354,0.021714,0.029956,0.052278,0.034082,0.044483,0.065762,0.051314,0.030251,0.024786,0.051017,0.036792,0.024224,0.034938,0.054123,0.031394,0.057233,0.029540,0.043052,0.037520,0.040505,0.016974
BNK금융지주,0.016259,0.028559,0.018321,0.083733,0.012473,0.014098,0.023040,0.022257,0.018935,0.012164,0.008501,0.018494,0.019030,0.015582,0.011775,0.003563,0.009113,0.014541,0.009577,0.011417,0.015587,0.032043,0.030727,0.020006,0.046495,0.011510,0.032355,0.018298,0.021880,0.017361,0.015402,0.014663,0.013263,0.005868,0.018479,0.016354,0.022861,0.023508,0.033067,0.026055,...,0.028589,0.009484,0.027168,0.019119,0.018103,0.010003,0.014708,0.012246,0.020902,0.017688,0.018922,0.011041,0.019956,0.023553,0.022307,0.021829,0.019787,0.009810,0.017787,0.014387,0.009679,0.017825,0.023201,0.014209,0.016213,0.010575,0.020671,0.014650,0.019305,0.011116,0.016028,0.015402,0.013333,0.018591,0.021817,0.025214,0.021398,0.020180,0.017324,0.006361
BYC,0.020442,0.013711,0.022236,0.012473,0.066196,0.013008,0.014710,0.018994,0.011472,0.006798,0.010731,0.009914,0.017913,0.014471,0.012357,0.003271,0.017627,0.015473,0.009824,0.013221,0.007809,0.016338,0.009639,0.016779,0.011088,0.014075,0.016467,0.014649,0.020590,0.016975,0.019937,0.011141,0.012749,0.006429,0.015252,0.011525,0.014479,0.021820,0.015605,0.015877,...,0.021864,0.009498,0.008542,0.008648,0.019583,0.018662,0.015814,0.013930,0.012115,0.014587,0.015951,0.023975,0.013918,0.019141,0.017449,0.018714,0.019545,0.012802,0.015244,0.011964,0.012366,0.016829,0.011030,0.010637,0.015116,0.018924,0.012568,0.011053,0.021227,0.010930,0.012244,0.014159,

In [43]:
S.shape

(1476, 1476)

In [49]:
!pip install cvxpy

In [55]:
# Optimize for the maximal Sharpe ratio
# 💛데이터셋이 너무 많으면, ef.max_sharpe()에서 에러남 -> solver를 SCS로 바꿔줌
# Rober says: 100개 이하로 종목을 추린 후에 실행시키기를 추천함 !
ef = EfficientFrontier(mu, S, solver="SCS")  # Create the Efficient Frontier Object

# Maximize the Sharpe ratio, and get the raw weights
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

ef.portfolio_performance(verbose=True)

OrderedDict([('3S', 0.0), ('AK홀딩스', 0.0), ('APS홀딩스', 0.0), ('BNK금융지주', 0.0), ('BYC', 0.0), ('CBI', 0.0), ('CJ', 0.0), ('CJ CGV', 0.0), ('CJ ENM', 0.0), ('CJ대한통운', 0.0), ('CJ씨푸드', 0.0), ('CJ제일제당', 0.0), ('CJ프레시웨이', 0.0), ('CMG제약', 0.0), ('CNH', 0.0), ('CNT85', 0.0), ('COWON', 0.0), ('CS', 0.0), ('CSA 코스믹', 0.0), ('CS홀딩스', 0.0), ('DB', 0.0), ('DB금융투자', 0.0), ('DB손해보험', 0.0), ('DB하이텍', 0.0), ('DGB금융지주', 0.0), ('DI동일', 0.0), ('DL', 0.0), ('DL건설', 0.01595), ('DMS', 0.0), ('DRB동일', 0.0), ('DSR제강', 0.0), ('E1', 0.0), ('EG', 0.0), ('EMW', 0.0), ('ES큐브', 0.0), ('F&F홀딩스', 0.01117), ('GKL', 0.0), ('GST', 0.0), ('GS건설', 0.0), ('GS글로벌', 0.0), ('GS리테일', 0.0), ('GV', 0.0), ('HB테크놀러지', 0.0), ('HDC', 0.0), ('HDC현대EP', 0.0), ('HMM', 0.0), ('HRS', 0.0), ('HSD엔진', 0.0), ('IHQ', 0.0), ('ISC', 0.0), ('ITX-AI', 0.0), ('JW신약', 0.0), ('JW중외제약', 0.0), ('JW홀딩스', 0.0), ('JYP Ent.', 0.0), ('KBI메탈', 0.0), ('KB금융', 0.0), ('KB오토시스', 0.0), ('KCC', 0.0), ('KCC건설', 0.0), ('KCI', 0.0), ('KCTC', 0.0), ('KC그린홀딩스', 0.0), ('

(0.28876802253445577, 0.12040463155720918, 2.2322066772552103)

In [56]:
weights

OrderedDict([('3S', 6.67702695e-08),
             ('AK홀딩스', -5.05510866e-08),
             ('APS홀딩스', -4.073610503e-07),
             ('BNK금융지주', 9.193961781e-07),
             ('BYC', 3.750789046e-07),
             ('CBI', 4.662768522e-07),
             ('CJ', 1.784672276e-07),
             ('CJ CGV', 1.3996549353e-06),
             ('CJ ENM', -8.736482439e-07),
             ('CJ대한통운', -1.524856696e-06),
             ('CJ씨푸드', 1.0293363634e-06),
             ('CJ제일제당', -2.14504981e-07),
             ('CJ프레시웨이', 1.189814069e-07),
             ('CMG제약', -1.274605619e-07),
             ('CNH', -6.810159263e-07),
             ('CNT85', 9.1246024e-09),
             ('COWON', 7.77433994e-08),
             ('CS', 5.29007312e-08),
             ('CSA 코스믹', -5.563413925e-07),
             ('CS홀딩스', 8.943831257e-07),
             ('DB', 1.626011034e-07),
             ('DB금융투자', 4.632684237e-07),
             ('DB손해보험', 1.8070076927e-06),
             ('DB하이텍', 7.49063975e-07),
             ('DGB

In [57]:
cleaned_weights

OrderedDict([('3S', 0.0),
             ('AK홀딩스', 0.0),
             ('APS홀딩스', 0.0),
             ('BNK금융지주', 0.0),
             ('BYC', 0.0),
             ('CBI', 0.0),
             ('CJ', 0.0),
             ('CJ CGV', 0.0),
             ('CJ ENM', 0.0),
             ('CJ대한통운', 0.0),
             ('CJ씨푸드', 0.0),
             ('CJ제일제당', 0.0),
             ('CJ프레시웨이', 0.0),
             ('CMG제약', 0.0),
             ('CNH', 0.0),
             ('CNT85', 0.0),
             ('COWON', 0.0),
             ('CS', 0.0),
             ('CSA 코스믹', 0.0),
             ('CS홀딩스', 0.0),
             ('DB', 0.0),
             ('DB금융투자', 0.0),
             ('DB손해보험', 0.0),
             ('DB하이텍', 0.0),
             ('DGB금융지주', 0.0),
             ('DI동일', 0.0),
             ('DL', 0.0),
             ('DL건설', 0.01595),
             ('DMS', 0.0),
             ('DRB동일', 0.0),
             ('DSR제강', 0.0),
             ('E1', 0.0),
             ('EG', 0.0),
             ('EMW', 0.0),
             ('ES큐브', 0.0),


In [58]:
!pip install pulp

     |████████████████████████████████| 40.6MB 74kB/s 


In [81]:
# Get the discrete allocation of each sharpe per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

# 투자금액 (단위: KRW)
portfolio_val = 5000000

latest_prices = get_latest_prices(df2)

weights = cleaned_weights

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=portfolio_val)

allocation, leftover = da.lp_portfolio()

print('Discrete Allocaion: ', allocation)
print('Funds Remaining: ', leftover, ' KRW')

Discrete Allocaion:  {'DL건설': 2, 'F&F홀딩스': 2, 'MH에탄올': 1, 'NICE평가정보': 2, '고려신용정보': 31, '국일제지': 36, '까뮤이앤씨': 25, '나이스정보통신': 2, '네이처셀': 4, '대성홀딩스': 10, '대한방직': 2, '동남합성': 4, '동원개발': 3, '동화기업': 1, '리노공업': 1, '맥쿼리인프라': 45, '메리츠금융지주': 5, '미원상사': 3, '부산가스': 1, '빅텍': 12, '삼성공조': 1, '쌍용C&E': 2, '아난티': 3, '에이치엘비': 1, '에이치엘비생명과학': 3, '에이텍': 2, '우리기술투자': 5, '이크레더블': 4, '지어소프트': 1, '진원생명과학': 1, '카카오': 1, '티씨케이': 1, '파라텍': 3, '포스코강판': 1, '푸른저축은행': 10, '한국가구': 6, '한국기업평가': 3, '한솔케미칼': 1}
Funds Remaining:  685.0  KRW


In [82]:
allocation

{'DL건설': 2,
 'F&F홀딩스': 2,
 'MH에탄올': 1,
 'NICE평가정보': 2,
 '고려신용정보': 31,
 '국일제지': 36,
 '까뮤이앤씨': 25,
 '나이스정보통신': 2,
 '네이처셀': 4,
 '대성홀딩스': 10,
 '대한방직': 2,
 '동남합성': 4,
 '동원개발': 3,
 '동화기업': 1,
 '리노공업': 1,
 '맥쿼리인프라': 45,
 '메리츠금융지주': 5,
 '미원상사': 3,
 '부산가스': 1,
 '빅텍': 12,
 '삼성공조': 1,
 '쌍용C&E': 2,
 '아난티': 3,
 '에이치엘비': 1,
 '에이치엘비생명과학': 3,
 '에이텍': 2,
 '우리기술투자': 5,
 '이크레더블': 4,
 '지어소프트': 1,
 '진원생명과학': 1,
 '카카오': 1,
 '티씨케이': 1,
 '파라텍': 3,
 '포스코강판': 1,
 '푸른저축은행': 10,
 '한국가구': 6,
 '한국기업평가': 3,
 '한솔케미칼': 1}

In [83]:
len(allocation)

38

In [84]:
# 포트폴리오에 포함된 종목을 리스트로 만들기
company_name = list(allocation)

In [85]:
company_name

['DL건설',
 'F&F홀딩스',
 'MH에탄올',
 'NICE평가정보',
 '고려신용정보',
 '국일제지',
 '까뮤이앤씨',
 '나이스정보통신',
 '네이처셀',
 '대성홀딩스',
 '대한방직',
 '동남합성',
 '동원개발',
 '동화기업',
 '리노공업',
 '맥쿼리인프라',
 '메리츠금융지주',
 '미원상사',
 '부산가스',
 '빅텍',
 '삼성공조',
 '쌍용C&E',
 '아난티',
 '에이치엘비',
 '에이치엘비생명과학',
 '에이텍',
 '우리기술투자',
 '이크레더블',
 '지어소프트',
 '진원생명과학',
 '카카오',
 '티씨케이',
 '파라텍',
 '포스코강판',
 '푸른저축은행',
 '한국가구',
 '한국기업평가',
 '한솔케미칼']

In [86]:
len(company_name)

38

In [87]:
# Get the discrete allocation values
discrete_allocation_list = []
for symbol in allocation:
  discrete_allocation_list.append(allocation.get(symbol))

In [88]:
discrete_allocation_list

[2,
 2,
 1,
 2,
 31,
 36,
 25,
 2,
 4,
 10,
 2,
 4,
 3,
 1,
 1,
 45,
 5,
 3,
 1,
 12,
 1,
 2,
 3,
 1,
 3,
 2,
 5,
 4,
 1,
 1,
 1,
 1,
 3,
 1,
 10,
 6,
 3,
 1]

In [89]:
# Create a dataframe for the portfolio
portfolio_df = pd.DataFrame(columns = ['Company_name', 'company_Ticker', 'Discrete_val_'+str(portfolio_val)])
portfolio_df

,Company_name,company_Ticker,Discrete_val_5000000


In [90]:
portfolio_df['Company_name'] = company_name
portfolio_df['company_Ticker'] = allocation
portfolio_df['Discrete_val_'+str(portfolio_val)] = discrete_allocation_list

In [91]:
# Show the portfolio
portfolio_df

,Company_name,company_Ticker,Discrete_val_5000000
0,DL건설,DL건설,2
1,F&F홀딩스,F&F홀딩스,2
2,MH에탄올,MH에탄올,1
3,NICE평가정보,NICE평가정보,2
4,고려신용정보,고려신용정보,31
5,국일제지,국일제지,36
6,까뮤이앤씨,까뮤이앤씨,25
7,나이스정보통신,나이스정보통신,2
8,네이처셀,네이처셀,4
9,대성홀딩스,대성홀딩스,10


In [92]:
# Show Funds Remaining
print('Funds Remaining: ', leftover, ' KRW')

# Show Portfolio performance
ef.portfolio_performance(verbose=True)

Funds Remaining:  685.0  KRW
Expected annual return: 28.9%
Annual volatility: 12.0%
Sharpe Ratio: 2.23


(0.28876802253445577, 0.12040463155720918, 2.2322066772552103)